# LaLiga Scouting & Performance Analysis (BigQuery) — Walkthrough Notebook
**Author / Autor:** Osval  
**Target environment / Entorno objetivo:** Google Colab + BigQuery

---

## What this is / Qué es
**EN**  
This notebook is a narrative walkthrough of the SQL pipeline used in this project. It includes both explanations (Markdown) and executable steps (Python + BigQuery).

**ES**  
Este notebook es un recorrido narrativo del pipeline SQL usado en el proyecto. Incluye explicaciones (Markdown) y pasos ejecutables (Python + BigQuery).

> **Note / Nota:** GitHub renders notebooks for reading, but execution is done in Colab (or locally).


## 0) Setup / Configuración

**EN**  
This notebook queries BigQuery tables. You need a Google account with access to a GCP project. Depending on your setup, creating tables may require billing to be enabled.

**ES**  
Este notebook consulta tablas en BigQuery. Necesitas una cuenta de Google con acceso a un proyecto de GCP. Según tu configuración, crear tablas puede requerir que la facturación esté habilitada.


In [2]:
# Install dependencies (Colab-friendly)
!pip -q install google-cloud-bigquery pydata-google-auth pandas

from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

# ---  (project/dataset/table names) ---
PROJECT_ID = "proyecto-laliga-24-25"   # e.g. "my-gcp-project"
DATASET_ID = "Laliga_24_25"            # e.g. "laliga_scouting"
RAW_TABLE  = "LaLiga_24_25"            # e.g. "players_raw"

RAW_FQN = f"{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE}"
STG_FQN = f"{PROJECT_ID}.{DATASET_ID}.stg_player_match"
MART_PLAYER_POS_FQN = f"{PROJECT_ID}.{DATASET_ID}.mart_player_season_by_position"
MART_TEAM_FQN = f"{PROJECT_ID}.{DATASET_ID}.mart_team_season"
MART_ENRICHED_FQN = f"{PROJECT_ID}.{DATASET_ID}.mart_player_season_enriched"
MART_FORM_FQN = f"{PROJECT_ID}.{DATASET_ID}.mart_player_form_last5"

client = bigquery.Client(project=PROJECT_ID)

def q(sql: str) -> pd.DataFrame:

    return client.query(sql).to_dataframe()

print("Configured RAW table:", RAW_FQN)


Configured RAW table: proyecto-laliga-24-25.Laliga_24_25.LaLiga_24_25


## 1) Quick profiling (like `df.info()` + `df.head()`)

**EN — Why**  
Before building marts, I check table scale and schema to avoid wrong assumptions.

**ES — Por qué**  
Antes de construir tablas analíticas, reviso tamaño y esquema para evitar supuestos erróneos.


In [3]:
# 1.1 Row count
q(f"SELECT COUNT(*) AS total_rows FROM `{RAW_FQN}`")


,total_rows
0,4658


In [4]:
# 1.2 Schema inspection (names + types)
q(f'''
SELECT column_name, data_type, is_nullable
FROM `{PROJECT_ID}.{DATASET_ID}.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = '{RAW_TABLE}'
ORDER BY ordinal_position
''')


,column_name,data_type,is_nullable
0,fecha_p,DATE,YES
1,Jugador,STRING,YES
2,Equipo,STRING,YES
3,Pais,STRING,YES
4,Posicion,STRING,YES
5,edad,STRING,YES
6,minutos,INT64,YES
7,goles,INT64,YES
8,asistencias,INT64,YES
9,Tiros,INT64,YES


In [5]:
# 1.3 Sample rows
q(f"SELECT * FROM `{RAW_FQN}` LIMIT 10")


,fecha_p,Jugador,Equipo,Pais,Posicion,edad,minutos,goles,asistencias,Tiros,Tiros_puerta,Amarillas,rojas,toques,regates,regates_int,regates_ok,entradas,blocajes,crea_t,crea_g,pase_compl,pase_int,pases_prog,conducc,conducc_prog,xg,npxg,xag,pen,pen_oj
0,2024-09-29,Abde Ezzalzouli,Real Betis,MAR,LW,22-287,87,0,0,2,2,0,0,47,0,6,0,0,1,9,0,19,31,3,32,5,1.2,0.4,0.4,1,0
1,2024-09-13,Abde Ezzalzouli,Real Betis,MAR,LW,22-271,90,1,0,4,4,0,0,71,0,4,1,0,0,2,1,39,55,6,40,9,1.2,1.2,0.0,0,0
2,2024-08-17,Aimar Oroz,Osasuna,ESP,LW,22-264,89,0,0,4,1,0,0,58,2,6,2,2,2,4,0,38,42,6,30,1,0.7,0.7,0.1,0,0
3,2024-09-30,Alex Baena,Villarreal,ESP,"LM,FW",23-072,45,1,1,2,1,0,0,40,0,1,0,2,0,6,1,23,31,6,21,1,1.1,0.3,0.6,1,0
4,2024-10-20,Alexander Sørloth,Atlético Madrid,NOR,FW,28-320,90,2,0,7,3,0,0,38,1,2,2,0,0,4,0,15,20,1,18,3,2.0,2.0,0.0,0,0
5,2024-11-04,Anastasios Douvikas,Celta Vigo,GRE,FW,25-094,66,1,0,3,2,1,0,25,1,1,0,0,0,1,1,8,15,0,11,0,1.3,1.3,0.0,0,0
6,2024-11-02,Ante Budimir,Osasuna,CRO,FW,33-103,84,1,0,7,0,0,0,34,0,0,0,0,2,5,1,10,17,6,18,1,1.4,0.7,0.2,1,1
7,2024-09-28,Ante Budimir,Osasuna,CRO,FW,33-068,76,2,0,2,1,0,0,22,0,1,0,0,2,1,1,9,11,0,13,1,1.0,0.2,0.0,1,1
8,2024-09-15,Antoine Griezmann,Atlético Madrid,FRA,"FW,CM",33-178,75,1,0,2,1,0,0,61,1,1,1,1,2,3,1,33,48,6,31,3,0.7,0.7,0.0,0,0
9,2024-09-14,Ayoze Pérez,Villarreal,ESP,FW,31-047,90,1,0,4,1,0,0,35,0,3,2,0,0,6,0,18,22,3,28,5,0.7,0.7,0.3,0,0


## 2) Staging layer: clean + typed table (`stg_player_match`)

**EN — Why**  
Raw CSV/JSON often stores numbers as strings and may contain whitespace or malformed values.  
This project uses defensive SQL: `SAFE_CAST`, `TRIM`, `NULLIF`, `SAFE_DIVIDE`.

**ES — Por qué**  
Los CSV/JSON suelen traer números como texto y valores inconsistentes.  
Este proyecto usa SQL defensivo: `SAFE_CAST`, `TRIM`, `NULLIF`, `SAFE_DIVIDE`.

**Grain / Grano**  
One row = player + team + date. Player names repeat across dates by design.  
Una fila = jugador + equipo + fecha. Los nombres se repiten por diseño.


In [6]:
# Create/replace staging table (adjust column names if your raw schema differs)
stg_sql = f'''
CREATE OR REPLACE TABLE `{STG_FQN}` AS
SELECT
  SAFE_CAST(`fecha_p` AS DATE) AS match_date,
  TRIM(CAST(`Jugador` AS STRING)) AS player_name,
  TRIM(CAST(`Equipo` AS STRING)) AS team,
  TRIM(CAST(`Pais` AS STRING)) AS nation,
  TRIM(CAST(`Posicion` AS STRING)) AS position,

  SAFE_CAST(`minutos` AS INT64) AS minutes,
  SAFE_CAST(`goles` AS INT64) AS goals,
  SAFE_CAST(`asistencias` AS INT64) AS assists,

  SAFE_CAST(`Tiros` AS INT64) AS shots_total,
  SAFE_CAST(`Tiros_puerta` AS INT64) AS shots_on_target,

  SAFE_CAST(`xg` AS FLOAT64) AS xg,
  SAFE_CAST(`xAG` AS FLOAT64) AS xag,

  TO_HEX(MD5(CONCAT(
    CAST(SAFE_CAST(`fecha_p` AS DATE) AS STRING), '|',
    TRIM(CAST(`Equipo` AS STRING)), '|',
    TRIM(CAST(`Jugador` AS STRING))
  ))) AS player_team_date_key
FROM `{RAW_FQN}`
WHERE `Jugador` IS NOT NULL
  AND `Equipo` IS NOT NULL
  AND `fecha_p` IS NOT NULL
'''
client.query(stg_sql).result()
print("Created:", STG_FQN)


Created: proyecto-laliga-24-25.Laliga_24_25.stg_player_match


## 3) Data Quality checks (nulls, duplicates)

**EN — Why**  
Quality is not assumed; it is measured. These checks validate the staging assumptions.

**ES — Por qué**  
La calidad no se asume; se mide. Estos checks validan los supuestos del staging.


In [7]:
# 3.1 Null counts on critical fields
q(f'''
SELECT
  COUNT(*) AS rows_total,
  COUNTIF(match_date IS NULL) AS null_match_date,
  COUNTIF(player_name IS NULL OR player_name = '') AS null_player_name,
  COUNTIF(team IS NULL OR team = '') AS null_team,
  COUNTIF(position IS NULL OR position = '') AS null_position
FROM `{STG_FQN}`
''')


,rows_total,null_match_date,null_player_name,null_team,null_position
0,4658,0,0,0,0


In [8]:
# 3.2 Duplicates by grain key (true duplicates)
q(f'''
SELECT player_team_date_key, COUNT(*) AS cnt
FROM `{STG_FQN}`
GROUP BY 1
HAVING COUNT(*) > 1
ORDER BY cnt DESC
LIMIT 50
''')


,player_team_date_key,cnt


## 4) Core mart: player season by position

**EN — Why**  
Scouting is role-dependent. We aggregate by player + team + nation + position.  
Per-90 metrics make players comparable despite different minutes.

**ES — Por qué**  
El scouting depende del rol. Agregamos por jugador + equipo + país + posición.  
Las métricas por 90 permiten comparar jugadores con minutos distintos.


In [9]:
mart_player_sql = f'''
CREATE OR REPLACE TABLE `{MART_PLAYER_POS_FQN}` AS
SELECT
  player_name,
  team,
  nation,
  position,

  COUNT(*) AS appearances_rows,
  COUNTIF(minutes > 0) AS matches_played,
  SUM(minutes) AS minutes,

  SUM(goals) AS goals,
  SUM(assists) AS assists,
  SUM(xg) AS xg,
  SUM(xag) AS xag,
  SUM(shots_total) AS shots_total,
  SUM(shots_on_target) AS shots_on_target,

  ROUND(SUM(goals) * 90.0 / NULLIF(SUM(minutes), 0), 3) AS goals_per90,
  ROUND(SUM(assists) * 90.0 / NULLIF(SUM(minutes), 0), 3) AS assists_per90,
  ROUND(SUM(xg) * 90.0 / NULLIF(SUM(minutes), 0), 3) AS xg_per90,
  ROUND(SUM(xag) * 90.0 / NULLIF(SUM(minutes), 0), 3) AS xag_per90,

  ROUND(SAFE_DIVIDE(SUM(goals), NULLIF(SUM(shots_on_target), 0)), 3) AS goals_per_shot_on_target
FROM `{STG_FQN}`
GROUP BY player_name, team, nation, position
'''
client.query(mart_player_sql).result()
print("Created:", MART_PLAYER_POS_FQN)


Created: proyecto-laliga-24-25.Laliga_24_25.mart_player_season_by_position


In [10]:
# Top players by attacking threat (xG+xAG per90) with minutes threshold
q(f'''
SELECT
  position,
  player_name,
  team,
  minutes,
  xg_per90,
  xag_per90,
  ROUND(xg_per90 + xag_per90, 3) AS threat_per90
FROM `{MART_PLAYER_POS_FQN}`
WHERE minutes >= 900
ORDER BY threat_per90 DESC
LIMIT 25
''')


,position,player_name,team,minutes,xg_per90,xag_per90,threat_per90
0,FW,Robert Lewandowski,Barcelona,1248,1.017,0.166,1.183
1,FW,Kylian Mbappé,Real Madrid,1019,0.821,0.071,0.892
2,RW,Lamine Yamal,Barcelona,1033,0.357,0.505,0.862
3,FW,Ante Budimir,Osasuna,1103,0.530,0.049,0.579
4,FW,Vedat Muriqi,Mallorca,901,0.390,0.060,0.450
5,FW,Isaac Romero,Sevilla,978,0.193,0.110,0.303
6,LB,Sergi Cardona,Villarreal,1232,0.124,0.124,0.248
7,CM,Daniel Parejo,Villarreal,924,0.185,0.029,0.214
8,CB,Nayef Aguerd,Real Sociedad,919,0.196,0.010,0.206
9,RB,Pablo Maffeo,Mallorca,916,0.020,0.177,0.197


## 5) Team context (join example)

**EN — Why**  
Team context helps interpret individual stats (dependency, style effects).

**ES — Por qué**  
El contexto del equipo ayuda a interpretar estadísticas individuales (dependencia, efectos de estilo).


In [11]:
team_sql = f'''
CREATE OR REPLACE TABLE `{MART_TEAM_FQN}` AS
SELECT
  team,
  SUM(minutes) AS team_minutes,
  SUM(goals) AS team_goals,
  SUM(xg) AS team_xg,
  ROUND(SUM(goals) * 90.0 / NULLIF(SUM(minutes), 0), 3) AS team_goals_per90
FROM `{STG_FQN}`
GROUP BY team
'''
client.query(team_sql).result()
print("Created:", MART_TEAM_FQN)


Created: proyecto-laliga-24-25.Laliga_24_25.mart_team_season


In [12]:
enrich_sql = f'''
CREATE OR REPLACE TABLE `{MART_ENRICHED_FQN}` AS
SELECT
  p.*,
  t.team_goals_per90,
  ROUND(SAFE_DIVIDE(p.goals, NULLIF(t.team_goals, 0)), 4) AS share_of_team_goals
FROM `{MART_PLAYER_POS_FQN}` p
LEFT JOIN `{MART_TEAM_FQN}` t
USING (team)
'''
client.query(enrich_sql).result()
print("Created:", MART_ENRICHED_FQN)


Created: proyecto-laliga-24-25.Laliga_24_25.mart_player_season_enriched


In [13]:
# Players with high share of team goals (minimum minutes)
q(f'''
SELECT
  player_name, team, position, minutes, goals, team_goals_per90, share_of_team_goals
FROM `{MART_ENRICHED_FQN}`
WHERE minutes >= 900
ORDER BY share_of_team_goals DESC
LIMIT 25
''')


,player_name,team,position,minutes,goals,team_goals_per90,share_of_team_goals
0,Ante Budimir,Osasuna,FW,1103,9,0.109,0.5000
1,Robert Lewandowski,Barcelona,FW,1248,15,0.273,0.3125
2,Kylian Mbappé,Real Madrid,FW,1019,7,0.188,0.2258
3,Vedat Muriqi,Mallorca,FW,901,3,0.086,0.2000
4,César Tárrega,Valencia,CB,990,2,0.091,0.1538
5,Abdul Mumin,Rayo Vallecano,CB,993,1,0.071,0.1000
6,Daniel Vivian,Athletic Club,CB,1045,2,0.137,0.0833
7,Lamine Yamal,Barcelona,RW,1033,4,0.273,0.0833
8,Daniel Parejo,Villarreal,CM,924,2,0.163,0.0800
9,Marash Kumbulla,Espanyol,CB,1012,1,0.091,0.0714


## 6) Recent form (window functions)

**EN — Why**  
Season totals hide short-term dynamics. Rolling windows approximate recent form (last 5 match dates).

**ES — Por qué**  
Los totales esconden dinámicas a corto plazo. Ventanas móviles aproximan la forma reciente (últimas 5 fechas).


In [14]:
form_sql = f'''
CREATE OR REPLACE TABLE `{MART_FORM_FQN}` AS
WITH base AS (
  SELECT
    player_name,
    team,
    match_date,
    minutes,
    goals,
    xg,
    SUM(goals) OVER (PARTITION BY player_name, team ORDER BY match_date ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS goals_last5,
    SUM(xg) OVER (PARTITION BY player_name, team ORDER BY match_date ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS xg_last5,
    SUM(minutes) OVER (PARTITION BY player_name, team ORDER BY match_date ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS minutes_last5
  FROM `{STG_FQN}`
)
SELECT
  player_name,
  team,
  match_date,
  goals_last5,
  xg_last5,
  minutes_last5,
  ROUND(goals_last5 * 90.0 / NULLIF(minutes_last5,0), 3) AS goals_per90_last5,
  ROUND(xg_last5 * 90.0 / NULLIF(minutes_last5,0), 3) AS xg_per90_last5
FROM base
'''
client.query(form_sql).result()
print("Created:", MART_FORM_FQN)


Created: proyecto-laliga-24-25.Laliga_24_25.mart_player_form_last5


In [15]:
# Example: players trending up in last 5 (minimum rolling minutes)
q(f'''
SELECT
  player_name, team, match_date, minutes_last5, goals_per90_last5, xg_per90_last5
FROM `{MART_FORM_FQN}`
WHERE minutes_last5 >= 300
ORDER BY goals_per90_last5 DESC
LIMIT 25
''')


,player_name,team,match_date,minutes_last5,goals_per90_last5,xg_per90_last5
0,Robert Lewandowski,Barcelona,2024-10-20,390,1.846,1.638
1,Robert Lewandowski,Barcelona,2024-10-26,390,1.846,1.431
2,Robert Lewandowski,Barcelona,2024-11-03,404,1.559,1.248
3,Ayoze Pérez,Villarreal,2024-10-26,350,1.543,0.694
4,Robert Lewandowski,Barcelona,2024-11-10,425,1.482,1.101
5,Robert Lewandowski,Barcelona,2024-10-06,393,1.374,1.260
6,Ayoze Pérez,Villarreal,2024-11-09,358,1.257,0.654
7,Giovani Lo Celso,Real Betis,2024-09-29,368,1.223,0.514
8,Ayoze Pérez,Villarreal,2024-09-26,374,1.203,0.505
9,Raphinha,Barcelona,2024-09-28,392,1.148,0.941
